In [28]:
import pandas as pd
import pandas as pd
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Masking, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import model_from_json

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import sys

In [18]:
content = pd.read_csv("Learn2rank/datal2r/abstracts2018.csv", encoding='latin-1').iloc[:,1:]
id_data = pd.read_csv("Learn2rank/datal2r/abstracts.judgments.2018.csv", encoding='latin-1')
data = pd.merge(id_data, content, how='right', left_on="trec_doc_id", right_on="ID").iloc[:, [0, -2,-1]]

In [88]:
#data.head()
data = data.rename(columns={"trec_topic_number": "ID"}).sort_values(by="ID")

NameError: name 'data' is not defined

In [87]:
import csv 
import requests 
import xml.etree.ElementTree as ET 
import pandas as pd
def parsegene(xmlfile): 
    # create element tree object 
    tree = ET.parse(xmlfile) 
    # get root element 
    root = tree.getroot()
    # create empty list for news items 
    geneitems = [] 
    # iterate news items 
    for item in root.findall('./topic/gene'): 
        gene = item.text
        # append news dictionary to news items list 
        geneitems.append(gene)  
    # return news items list 
    for i in range(19,21):
        geneitems.remove(geneitems[i])
    genes = str(geneitems)
    genes = re.sub("[a-x]|\[|\]|\'|\,|\(|\)|\W\d+\W|[y].","", genes).split(" ")
    geneslist = []                
    for i in genes:
        if i != "":
            geneslist.append(i)
    pattern = "|".join(geneslist)
    return pattern

df = data.dropna()
nltk.download('stopwords')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
def clean_text(text):
    genes = parsegene("Topics/topics2018.xml")
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    # Clean the text
    text = re.sub(r"\[\D\S{6}:\S{5}\]", "", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\\n*", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r'\,|\.|\:|\&', "", text)
    text = re.sub(r'\{|\}|\[|\]|\(|\)', "", text)
    text = re.sub(genes, "gene", text)
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = [wn.lemmatize(word) for word in text]
    text = " ".join(stemmed_words)
    return text
df['Abstracts'] = df['Abstracts'].map(lambda x: clean_text(x))

vocabulary_size = 20000
def process(df):
    tokenizer = Tokenizer(num_words= vocabulary_size)
    tokenizer.fit_on_texts(df['Abstracts'])
    sequences = tokenizer.texts_to_sequences(df['Abstracts'])
    data = pad_sequences(sequences, maxlen=500)
    labels = np.array(df["Class"])
    df_save = pd.DataFrame(data)
    df_save.head(10)
    df_label = pd.DataFrame(np.array(labels))
    print(df.head())
    df_id = pd.DataFrame(np.array(df["ID"]))
    datamat = pd.concat([df_save, df_label], axis = 1, ignore_index = True)
    datamat = pd.concat([df_id, datamat], axis = 1, ignore_index = True)
    return data, labels, datamat, tokenizer, labels
x, y, vecdf, tokenizer, labels = process(df)
from sklearn.model_selection import train_test_split
train, test = train_test_split(vecdf, test_size=0.2)
train_x = train.iloc[:,1:-1]
train_y = train.iloc[:,-1]
train_id = train.iloc[:,0]
test_x = test.iloc[:,1:-1]
test_y = test.iloc[:,-1]
test_id = test.iloc[:,0]

NameError: name 'data' is not defined

In [29]:
train_x.head()

,1,2,3,4,5,6,7,8,9,10,...,491,492,493,494,495,496,497,498,499,500
19314,0,0,0,0,0,0,0,0,0,0,...,463,86,173,24,463,506,8,1549,957,149
4860,0,0,0,0,0,0,0,0,0,0,...,8,77,31,1565,5109,1024,2332,817,119,1
14160,0,0,0,0,0,0,0,0,0,0,...,362,69,6,613,86,480,707,821,3294,4
4119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1031
20169,0,0,0,0,0,0,0,0,0,0,...,344,189,34,504,304,4576,682,949,55,476


In [82]:
import sklearn
x_train, y_train = sklearn.datasets.load_svmlight_file("data/train.txt")
x_valid, y_valid = sklearn.datasets.load_svmlight_file("data/val.txt")
x_test, y_test = sklearn.datasets.load_svmlight_file("data/test.txt")

In [83]:
q_train = np.loadtxt('data/train.group')
q_valid = np.loadtxt('data/val.group')
q_test = np.loadtxt('data/test.group')

In [84]:
import lightgbm as lgb
gbm = lgb.LGBMRanker()
bst = gbm.fit(x_train, y_train, group=q_train, eval_set=[(x_valid, y_valid)],
eval_group=[q_valid], eval_at=[1, 3], early_stopping_rounds=20, verbose=True,
callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)])

[1]	valid_0's ndcg@1: 0.346667	valid_0's ndcg@3: 0.358287
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's ndcg@1: 0.586667	valid_0's ndcg@3: 0.54946
[3]	valid_0's ndcg@1: 0.666667	valid_0's ndcg@3: 0.664841
[4]	valid_0's ndcg@1: 0.713333	valid_0's ndcg@3: 0.698436
[5]	valid_0's ndcg@1: 0.873333	valid_0's ndcg@3: 0.751956
[6]	valid_0's ndcg@1: 0.853333	valid_0's ndcg@3: 0.763874
[7]	valid_0's ndcg@1: 0.86	valid_0's ndcg@3: 0.790466
[8]	valid_0's ndcg@1: 0.893333	valid_0's ndcg@3: 0.801564
[9]	valid_0's ndcg@1: 0.866667	valid_0's ndcg@3: 0.814153
[10]	valid_0's ndcg@1: 0.906667	valid_0's ndcg@3: 0.817691
[11]	valid_0's ndcg@1: 0.88	valid_0's ndcg@3: 0.814301
[12]	valid_0's ndcg@1: 0.9	valid_0's ndcg@3: 0.821229
[13]	valid_0's ndcg@1: 0.886667	valid_0's ndcg@3: 0.824357
[14]	valid_0's ndcg@1: 0.886667	valid_0's ndcg@3: 0.828715
[15]	valid_0's ndcg@1: 0.9	valid_0's ndcg@3: 0.832588
[16]	valid_0's ndcg@1: 0.9	valid_0's ndcg@3: 0.830205
[17]	valid_0's ndcg@1: 0.88

In [86]:
from scipy.stats import spearmanr
preds_train = gbm.predict(x_train)
preds_train
spearmanr(y_train, preds_train)

SpearmanrResult(correlation=0.3908338387085487, pvalue=0.0)

In [57]:
preds_train.shape

(17223,)

In [79]:
preds_test = gbm.predict(x_test)
preds_test
spearmanr(y_test, preds_test)

SpearmanrResult(correlation=0.3112367809291524, pvalue=3.0967241053496516e-121)

In [81]:
preds_test.shape

(5383,)